In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Tariffinformasjon
tariffs = {
    '1ad': {'self': 61, 'helfo': 19, 'total': 80},
    '1e': {'self': 59, 'helfo': 10, 'total': 69},
    '701a': {'self': 61, 'helfo': 11, 'total': 72},
    '701c': {'self': 0, 'helfo': 24, 'total': 24},  # Takst for sendeprøver
    '705k': {'self': 0, 'helfo': 51, 'total': 51},
    '706f': {'self': 0, 'helfo': 76, 'total': 76},
    '708a': {'self': 0, 'helfo': 24, 'total': 24},
    '709': {'self': 0, 'helfo': 157, 'total': 157},
    '704j': {'self': 0, 'helfo': 39, 'total': 39},
    '710': {'self': 0, 'helfo': 85, 'total': 85},
    '706h': {'self': 0, 'helfo': 38, 'total': 38},
    '706k': {'self': 0, 'helfo': 47, 'total': 47},
    '706l': {'self': 0, 'helfo': 62, 'total': 62},
    '705h': {'self': 0, 'helfo': 19, 'total': 19},
    '705i': {'self': 0, 'helfo': 45, 'total': 45},
    '10c': {'self': 155, 'helfo': 0, 'total': 155},  # Ny takst for Glukosebelastningstest
    '121a': {'self': 0, 'helfo': 177, 'total': 177},  # Ny takst for Glukosebelastningstest
    '618': {'self': 0, 'helfo': 177, 'total': 177},  # Ny takst for overvåket urinprøvetaking
    'T+250': {'self': 0, 'helfo': 0, 'total': 0},  # Tilleggshonorar er en spesiell type
    'T+606': {'self': 0, 'helfo': 0, 'total': 0}  # Tilleggshonorar for politi
}

# Widget for valg av kontakttype
contact_type = widgets.RadioButtons(
    options=['Kun laboratorie', 'Ekstern rekvirent', 'Konsultasjon samme dag'],
    description='Velg kontakttype:'
)

# Prøvevalg med sjekkbokser for lokale prøver
test_to_tariff_keys = {
    'CRP': ['701a', '705k'],
    'Differensialtelling av leukocytter': ['701a', '706f'],
    'Glukose': ['701a', '708a'],
    'HbA1c': ['701a', '709'],
    'Hemoglobin': ['1e'],
    'INR': ['701a', '710'],
    'Leukocytter': ['1e'],
    'MCV': ['701a', '706f'],
    'Monospot': ['701a', '706l'],
    'Senkningsreaksjon': ['1e'],
    'Strep-A': ['701a', '706k'],
    'Trombocytter': ['701a', '706f'],
    'Urin-HCG': ['701a', '706h'],
    'Urin-stix': []
}

procedure_to_tariff_keys = {
    'Glukosebelastningstest': ['10c', '121a'],  # Flyttet til prosedyrer
    'Hemofec (på kontoret)': ['701a', '704j'],  # Flyttet til prosedyrer
    'Prøvetaking for klamydia': ['701a', '705h'],  # Flyttet til prosedyrer
    'Prøvetaking for sopp eller skabb': ['701a', '705i'],  # Flyttet til prosedyrer
    'Overvåket urinprøvetaking etter forespørsel fra spesialisthelsetjenesten': ['701a', '618'],  # Flyttet til prosedyrer
    'Prøvetaking for barnevernstjenesten': ['T+250'],  # Ny prøve under prosedyrer
    'Prøvetaking førerkort': ['T+250'],  # Ny prøve under prosedyrer
    'Prøvetaking for politi': ['T+606']  # Ny prøve under prosedyrer
}

local_tests_checkboxes = [widgets.Checkbox(value=False, description=test) for test in test_to_tariff_keys.keys()]
local_tests_label = widgets.Label('Lokale prøver:')

procedure_checkboxes = [widgets.Checkbox(value=False, description=test) for test in procedure_to_tariff_keys.keys()]
procedure_label = widgets.Label('Prosedyrer:')

# Sendeprøver med sjekkbokser
sendeprover_checkboxes = [
    widgets.Checkbox(value=False, description='Volda sykehus biokjemi (blodprøve)'),
    widgets.Checkbox(value=False, description='Volda sykehus biokjemi (urinprøve)'),
    widgets.Checkbox(value=False, description='Ålesund sykehus biokjemi (blodprøve)'),
    widgets.Checkbox(value=False, description='Ålesund sykehus (urinprøve)'),
    widgets.Checkbox(value=False, description='Ålesund sykehus (avføringsprøve)'),
    widgets.Checkbox(value=False, description='Ålesund sykehus (cytologi)'),
    widgets.Checkbox(value=False, description='Ålesund sykehus (histologi)'),
    widgets.Checkbox(value=False, description='Molde sykehus biokjemi (blodprøve)'),
    widgets.Checkbox(value=False, description='Molde sykehus biokjemi (urinprøve)'),
    widgets.Checkbox(value=False, description='Molde sykehus mikrobiologi (virusserologi)'),
    widgets.Checkbox(value=False, description='Molde sykehus mikrobiologi (PCR)'),
    widgets.Checkbox(value=False, description='Molde sykehus mikrobiologi (dyrkning)'),
    widgets.Checkbox(value=False, description='Kristiansund sykehus (avføringsprøve)'),
    widgets.Checkbox(value=False, description='St. Olavs hospital')
]
sendeprover_label = widgets.Label('Sendeprøver:')

# Beregningsknapp og output-område
calculate_button = widgets.Button(description="Beregn")
result_output = widgets.Output()

def calculate_button_clicked(b):
    sum_self = sum_helfo = sum_total = 0
    tilleggshonorar = 0
    selected_tariffs = []

    # Behandler lokale prøver
    for checkbox in local_tests_checkboxes:
        if checkbox.value:
            tariff_keys = test_to_tariff_keys[checkbox.description].copy()
            selected_tariffs.extend(tariff_keys)

    # Behandler prosedyrer
    for checkbox in procedure_checkboxes:
        if checkbox.value:
            tariff_keys = procedure_to_tariff_keys[checkbox.description].copy()
            if 'T+250' in tariff_keys:
                tilleggshonorar += 250
            if 'T+606' in tariff_keys:
                tilleggshonorar += 606
            selected_tariffs.extend(tariff_keys)

    # Regelen om at 121a ikke kan legges til hvis 708a er lagt til, og omvendt
    if '121a' in selected_tariffs and '708a' in selected_tariffs:
        selected_tariffs = [tariff for tariff in selected_tariffs if tariff != '708a']

    # Automatisk valg basert på prosedyrer
    if procedure_checkboxes[[cb.description for cb in procedure_checkboxes].index('Prøvetaking for klamydia')].value:
        sendeprover_checkboxes[[cb.description for cb in sendeprover_checkboxes].index('Molde sykehus mikrobiologi (PCR)')].value = True
    if procedure_checkboxes[[cb.description for cb in procedure_checkboxes].index('Prøvetaking for sopp eller skabb')].value:
        sendeprover_checkboxes[[cb.description for cb in sendeprover_checkboxes].index('Molde sykehus mikrobiologi (dyrkning)')].value = True

    # Automatisk valg basert på "Overvåket urinprøvetaking etter forespørsel fra spesialisthelsetjenesten"
    if procedure_checkboxes[[cb.description for cb in procedure_checkboxes].index('Overvåket urinprøvetaking etter forespørsel fra spesialisthelsetjenesten')].value:
        sendeprover_checkboxes[[cb.description for cb in sendeprover_checkboxes].index('St. Olavs hospital')].value = True

    # Behandler sendeprøver
    count_sendeprover = sum(checkbox.value for checkbox in sendeprover_checkboxes)
    if count_sendeprover > 0:
        selected_tariffs.append('701a')
    selected_tariffs.extend(['701c'] * (count_sendeprover - 1))

    # Legger til '1ad' dersom "Ekstern rekvirent" er valgt
    if contact_type.value == 'Ekstern rekvirent':
        selected_tariffs.append('1ad')

    # Unngå duplikater av '701a' og '1e'
    if '701a' in selected_tariffs:
        selected_tariffs = [tariff for tariff in selected_tariffs if tariff != '701a']
        selected_tariffs.append('701a')
    if '1e' in selected_tariffs:
        selected_tariffs = [tariff for tariff in selected_tariffs if tariff != '1e']
        selected_tariffs.append('1e')

    # Spesialhåndtering for '1e'
    if '701a' in selected_tariffs and '1e' in selected_tariffs:
        while '1e' in selected_tariffs:
            selected_tariffs.remove('1e')
    if contact_type.value != 'Kun laboratorie':
        while '1e' in selected_tariffs:
            selected_tariffs.remove('1e')

    # Beregner totale kostnader basert på valgte takster
    tariff_count = {}
    for tariff in selected_tariffs:
        if tariff in tariffs:
            sum_self += tariffs[tariff]['self']
            sum_helfo += tariffs[tariff]['helfo']
            sum_total += tariffs[tariff]['total']
            tariff_count[tariff] = tariff_count.get(tariff, 0) + 1

    # Legger til tilleggshonorar
    sum_total += tilleggshonorar

    with result_output:
        clear_output(wait=True)
        print(f"Sum egenbetaling: {sum_self} kr")
        print(f"Sum HELFO-refusjon: {sum_helfo} kr")
        print(f"Sum totalt: {sum_total} kr")
        print("Brukte takster:", ', '.join(f"{tariff} x {tariff_count[tariff]}" for tariff in tariff_count))
        if '1ad' in selected_tariffs:
            print("Husk å skrive hvem som er ekstern rekvirent i journal.")
        if 'Prøvetaking for barnevernstjenesten' in [cb.description for cb in procedure_checkboxes if cb.value]:
            print("Faktura skal sendes til barnevernstjenesten. Prøven skal sendes til Fürst. Barnevernstjenesten skal ordne rekvisisjon og stå som betaler av prøven.")
        if 'Prøvetaking førerkort' in [cb.description for cb in procedure_checkboxes if cb.value]:
            print("Pasienten skal betale selv for prøvetaking og analyse. Prøven skal sendes til Fürst. Fastlegen skal ordne rekvisisjon. Pasienten skal stå som fakturamottaker på rekvisisjonsskjema.")
        if 'Prøvetaking for politi' in [cb.description for cb in procedure_checkboxes if cb.value]:
            print("Faktura skal sendes til politiet. Politiet skal ha med rekvisisjonsskjema.")

calculate_button.on_click(calculate_button_clicked)

# Viser alle widgets
display(contact_type)
display(local_tests_label)
for checkbox in local_tests_checkboxes:
    display(checkbox)
display(procedure_label)
for checkbox in procedure_checkboxes:
    display(checkbox)
display(sendeprover_label)
for checkbox in sendeprover_checkboxes:
    display(checkbox)
display(calculate_button, result_output)


RadioButtons(description='Velg kontakttype:', options=('Kun laboratorie', 'Ekstern rekvirent', 'Konsultasjon s…

Label(value='Lokale prøver:')

Checkbox(value=False, description='CRP')

Checkbox(value=False, description='Differensialtelling av leukocytter')

Checkbox(value=False, description='Glukose')

...

Checkbox(value=False, description='Molde sykehus mikrobiologi (dyrkning)')

Checkbox(value=False, description='Kristiansund sykehus (avføringsprøve)')

Checkbox(value=False, description='St. Olavs hospital')

Button(description='Beregn', style=ButtonStyle())

Output()